In [13]:
using Pkg; Pkg.activate(".."); Pkg.instantiate()

using OceanTurb, Printf, Plots, Distributions, Statistics, Random, NetCDF, JLD2

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [14]:
# Setup
directory = "free_convection_N128_k1e-4_Float64"
prefix = "convection_"
top_flux = "75.0_"
bot_grad = "0.01_"
grid_points = "128_"

filename(iter) = joinpath(directory, prefix * top_flux * bot_grad * grid_points * lpad("$iter", 9, "0") * ".nc")

# Make filenames
ndata = 4
iters = [i*3600 for i = 1:ndata]
times = [iter*6 for iter in iters]
filenames = [filename(iter) for iter in iters]

4-element Array{String,1}:
 "free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000003600.nc"
 "free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000007200.nc"
 "free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000010800.nc"
 "free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000014400.nc"

In [2]:
T_data = Array{Float64, 1}[]

T₀ = 273.15

for filename in filenames
    T = ncread(filenames[i], "T") # 3D Oceananigans field
    
    # Take horizontal mean and convert to Celsius
    T_data_i = dropdims(mean(T, dims=[1, 2]), dims=[1, 2]) .- T₀
    T_data_i = reverse(T_data_i)
    
    push!(T_data, T_data_i)
end

# Get vertical grid
z = ncread(filenames[1], "zC")

["000000100", "000003600", "000007200", "000010800"]
free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000000100.nc
free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000003600.nc
free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000007200.nc
free_convection_N128_k1e-4_Float64/convection_75.0_0.01_128_000010800.nc


In [ ]:
# Save data
processsed_filename = "processed_data.jld2"

jldopen(filename, "w") do file
    file["z"] = z
    file["t_data"] = times
    file["T_data"] = T_data
end